In [14]:
import numpy as np
from numpy import sqrt, log, exp, pi
import pandas as pd
import scipy.stats
from scipy.stats import norm

from math import sqrt

import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
import matplotlib.pyplot as plt

from numpy import mean, absolute
import yfinance as yf


import warnings
warnings.filterwarnings("ignore")


In [15]:
# Coleta os dados

ticker1 = "PETR4.SA" 
df1 = yf.download(ticker1, "2012-01-01", "2023-12-31")
df1["Returns"] = df1["Adj Close"].pct_change(1)
df1["Vol"] = df1["Returns"].rolling(20).std()*np.sqrt(252)
df1.dropna(axis = 0, inplace = True) 
#print(df1.tail())

# Calcula os quantis (10)
quantiles = np.quantile(df1["Vol"], np.linspace(0, 1, 11))



[*********************100%***********************]  1 of 1 completed


In [16]:
quantiles

array([0.13756513, 0.24962211, 0.2880691 , 0.31778359, 0.35072547,
       0.381623  , 0.41139838, 0.46104505, 0.52958708, 0.65589746,
       1.98085642])

In [17]:
# Discretiza os dados em 10 quantis
quantile_labels = pd.cut(df1["Vol"], bins = quantiles, labels = ["I","II","III","IV","V"
                                                                 ,"VI","VII","VIII","IX","X"])
df1["Vol_Rank"] = quantile_labels

df1["Vol_Rank_"] = pd.cut(df1["Vol"], bins = quantiles, labels = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
df1.dropna(axis = 0, inplace = True) 
df1["Vol_Rank_"] = df1["Vol_Rank_"].astype(int)

print("Último Volatility Rank:", quantile_labels[-1])
print("Volatility Rank há 5 pregões:", quantile_labels[-5])
print("Volatility Rank há 10 pregões:", quantile_labels[-10])

Último Volatility Rank: III
Volatility Rank há 5 pregões: II
Volatility Rank há 10 pregões: II


In [18]:
df1.tail(15)

,Open,High,Low,Close,Adj Close,Volume,Returns,Vol,Vol_Rank,Vol_Rank_
Date,,,,,,,,,,
2023-07-14,29.500000,29.580000,28.860001,29.049999,29.049999,34451100,-0.019575,0.340576,IV,4
2023-07-17,28.410000,29.090000,27.950001,28.990000,28.990000,52160800,-0.002065,0.325174,IV,4
2023-07-18,29.000000,29.200001,28.709999,28.830000,28.830000,35640400,-0.005519,0.324247,IV,4
2023-07-19,28.809999,29.129999,28.570000,29.100000,29.100000,46517700,0.009365,0.282834,II,2
2023-07-20,29.070000,29.350000,28.879999,29.129999,29.129999,33177100,0.001031,0.281716,II,2
2023-07-21,29.150000,29.860001,29.129999,29.680000,29.680000,52059200,0.018881,0.255407,II,2
2023-07-24,29.799999,30.410000,29.760000,30.299999,30.299999,45268100,0.020889,0.253469,II,2
2023-07-25,30.500000,31.049999,30.200001,31.000000,31.000000,56261000,0.023102,0.265542,II,2
2023-07-26,30.950001,31.219999,30.629999,31.000000,31.000000,45880200,0.000000,0.263550,II,2


In [22]:
fig = make_subplots(rows = 2, cols = 1
                    , shared_xaxes = True
                    , vertical_spacing = 0.05)

fig.add_trace(go.Scatter(x = df1.index, y = df1["Adj Close"]
                                , name = "Close"
                                , line = dict(color = "blue"))
              , row = 1, col = 1)

fig.add_trace(go.Scatter(x = df1.index, y = df1["Vol_Rank_"]
                                , name = "Vol_Rank"
                                , line = dict(color = "red"))
              , row = 2, col = 1)


fig.update_layout(height = 800, width = 800
                  , title_text = "OM Studies: " + ticker1
                  , font_color = "blue"
                  , title_font_color = "black"
                  , xaxis_title = "Time"
                  , yaxis_title = ticker1
                  , yaxis2_title = "Vol Rank"
                  , font = dict(size = 15, color = "Black")
                 )

fig.update_layout(hovermode = "x")

# Code to exclude empty dates from the chart
dt_all = pd.date_range(start = df1.index[0]
                       , end = df1.index[-1]
                       , freq = "D")
dt_all_py = [d.to_pydatetime() for d in dt_all]
dt_obs_py = [d.to_pydatetime() for d in df1.index]

dt_breaks = [d for d in dt_all_py if d not in dt_obs_py]

fig.update_xaxes(
    rangebreaks = [dict(values = dt_breaks)]
)


fig.show()

In [25]:
fig = make_subplots(
    rows = 1, cols = 1
    , vertical_spacing = 0.1
    , horizontal_spacing = 0.1)

fig.add_trace(go.Indicator(
    domain = {"x": [0, 1], "y": [0, 1]}
    , value = df1["Vol_Rank_"].iloc[-1] #pega o valor mais recente
    , mode = "gauge+number+delta"
    , title = {"text": "Volatility Rank: " + ticker1, "font": {"size": 16, "color": "darkblue"}}
    , delta = {"reference":df1["Vol_Rank_"].iloc[-5], "increasing": {"color": "green"}
                                                    , "decreasing": {"color": "red"}} #compara com o valor do período anterior
    , gauge = {"axis": {"range": [None, df1["Vol_Rank_"].describe()[7]]} #intervalo total é de zero até a máxima de 2020
               , 'bar': {'color': "black"}
               , "steps" : [
                    {'range': [0, df1["Vol_Rank_"].describe()[1]], "color": "lightgreen"}
                    , {'range': [df1["Vol_Rank_"].describe()[1], df1["Vol_Rank_"].describe()[1]+2*df1["Vol_Rank_"].describe()[2]], 'color': "lightpink"}
                    , {'range': [df1["Vol_Rank_"].describe()[1]+2*df1["Vol_Rank_"].describe()[2], df1["Vol_Rank_"].describe()[7]], 'color': "red"}]
               , "threshold" : {'line': {'color': "black", 'width': 4}, 'thickness': 0.75
                            , 'value': df1["Vol_Rank_"].describe()[7]*0.98}})
            )

fig.update_layout(height = 600, width = 600
                  , title_text = "OM Studies: " + ticker1
                  , font_color = "blue"
                  , title_font_color = "black"
                  , font = dict(size = 15, color = "Black")
                 )